# Problem Set 2 - Naive Bayes, Decision Trees with ensemble methods
## CSCI 5622 - Fall 2021
***
**Name**: Alex Book
***

This assignment is due on Canvas by **3:55PM on October 7th**.

Submit only this Jupyter notebook to Canvas. Do not compress it using tar, rar, zip, etc.
Your solutions to analysis questions should be done in Markdown directly below the associated question.

Remember that you are encouraged to discuss the problems with your classmates and instructors, 
but **you must write all code and solutions on your own**, and list any people or sources consulted.
The only exception to this rule is that you may copy code directly from your own solution to homework 1.
***

## Overview 

Your task for this homework is to build a naive Bayes and a decision tree classifiers in the first 2 problems.
The last problem is about ensemble methods using scikit-learn decision tree as a weak learner.
We'll explore bagging, boosting (AdaBoost) and Random Forest.

In [1]:
import numpy as np
import matplotlib.pylab as plt
import pickle
from sklearn.metrics import precision_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from time import time
%matplotlib inline 

### Problem 1 - Naive Bayes [25 points]
***
Consider the problem of predicting whether a person has a college degree based on age, salary, and Colorado residency.
The dataset looks like the following.

|Age|Salary|Colorado Residency| College degree|
|:------:|:-----------:| :----------:|--:|
| 27 | 41,000 | Yes | Yes |
| 61 | 52,000 | No | No |
| 23 | 24,000 | Yes | No |
| 29 | 77,000 | Yes | Yes |
| 32 | 48,000 | No | Yes |
| 57 | 120,000 | Yes | Yes |
| 22 | 38,000 | Yes | Yes |
| 41 | 45,000 | Yes | No |
| 53 | 26,000 | No | No |
| 48 | 65,000 | Yes | Yes |


In [6]:
features = np.array([[27 , 41000 , 1],
              [61 , 52000 , 0],
              [23 , 24000 , 1],
              [29 , 77000 , 1],
              [32 , 48000 , 0],
              [57 , 120000 , 1],
              [22 , 38000 , 1],
              [41 , 45000 , 1],
              [53 , 26000 , 0],
              [48 , 65000 , 1]])
labels = np.array([1, 0, 0, 1, 1, 1, 1, 0, 0, 1])

1.1 What is our expected accuracy for the baseline case where we predict one label for all rows? (*2 points*)

#BEGIN Workspace 1.1

The expected accuracy for this baseline case is 60%, as we'll guess "yes".

#END Workspace 1.1

First, we have to find a way to deal with the continuous features. For now, let's put them into binary bins based on threshold arguments to our classifier - so we can treat this as a tuning parameter.

1.2 Complete `threshold_features` to convert age and salary features to binary ones using the threshold arguments. (*3 points*)

In [12]:
def threshold_features(features, age_threshold, salary_threshold):
    binary_features = features * 1 #This row just creates a "hard copy" of the X array so we can manipulate it as needed

    #BEGIN Workspace 1.2
    #TODO: Threshold the corresponding features
    
    for i in range(len(features)):
        binary_features[i][0] = 1 if features[i][0] > age_threshold else 0
        binary_features[i][1] = 1 if features[i][1] > salary_threshold else 0

    #END Workspace 1.2

    return binary_features

As seen during the class, given a row $(x_1, x_2, x_3)$, the naive Bayes classifier should assign the label $y$ that
maximizes:

\begin{align}
\log [p(y) \prod_i p(x_i | y)] = \log p(y) + \sum_{i} \log p(x_i | y)
\end{align}

$p(y)$ and $p(x_i | y)$ are computed using the training set (during `fit` call).

We have defined $p(x_i | y)$ as :
\begin{align}
p(x_i | y) = \frac{N_{y,i}}{N_y}
\end{align}
where $N_{y,i}$ is the number of rows where $y$ and $x_i$ occur together, and $N_y = \sum_i N_{y,i}$.

1.3 Complete the `fit` call by computing the counts and joint counts. Hint: Use `features_counts` to store the contingency
table $N_{y,i}$ for each feature $i$ and then use them to compute $\log p(x_i | y)$ (*10 points*)

1.4 Complete the `predict` call (*5 points*)

In [36]:
class NaiveBayes(object):
    """
    NaiveBayes classifier for binary features and binary labels
    """

    def __init__(self, alpha=0.0):
        self.alpha = alpha
        self.classes_counts = None
        self.features_counts = []
        self.classes_log_probabilities = None
        self.features_log_probabilities = [] # same structure as features_count

    def fit(self, X, y):
        """

        Parameters
        ----------
        X: binary np.array of shape (n_samples, n_features)
        y: corresponding labels of shape (n_samples,)
        Returns
        -------
        Trained classifier
        """

        #BEGIN Workspace 1.3

        #TODO: Compute the counts and joint counts
        
        n_samples, n_features = X.shape[0], X.shape[1]
        n_classes = np.unique(y).shape[0]
        
        self.classes_counts = np.zeros(n_classes)
        # 2 is the number of values that each feature can be
        self.features_counts = np.zeros((n_features, n_classes, 2))
        
        for i in range(n_samples):
            label = y[i]
            self.classes_counts[label] += 1
            for j in range(n_features):
                self.features_counts[j][label][X[i][j]] += 1
                
        self.features_log_probabilities = np.log2(self.features_counts / n_samples)
        self.classes_log_probabilities = np.log2(self.classes_counts / n_samples)
 
        #END Workspace 1.3

        return self

    def predict(self, x_test):
        joint_log_likelihood = np.zeros((x_test.shape[0], self.classes_counts.shape[0]))
        y_hat = 0
        #BEGIN Workspace 1.4
        #TODO: Find the corresponding labels using Naive bayes logic
        
        j_l_l = np.zeros(self.classes_counts.shape[0])
        for i in range(self.classes_counts.shape[0]):
            curr_prob = self.classes_log_probabilities[i]
            for j in range(self.features_counts.shape[0]):
                curr_prob += self.features_log_probabilities[j][i][x_test[j]]
            j_l_l[i] = curr_prob
        
        y_hat = np.argmax(j_l_l)
        
        #END Workspace 1.4
        return y_hat


1.5 Using age 30 and salary 40,000 as thresholds, transform the features and evaluate (accuracy) the NaiveBayes classifier
on the training data. (*5 points*)

In [37]:
clf = NaiveBayes()
#BEGIN Workspace 1.5
#TODO: Transform features to binary features, fit the classifier, report the accuracy
binary_features = threshold_features(features, 30, 40000)
clf.fit(binary_features, labels)

accuracy = 0
for i in range(len(binary_features)):
    prediction = clf.predict(binary_features[i])
    label = labels[i]
    if prediction == label:
        accuracy += 1/len(binary_features)
#     print(f"prediction: {clf.predict(binary_features[i])}, label: {labels[i]}")
    
print(f"Accuracy on training data: {accuracy*100}%")

#END Workspace 1.5

Accuracy on training data: 70.0%


**Bonus question** 1.6 Use the attribute `alpha` of the NaiveBayes to convert it to the smoothed NaiveBayes presented during the class. (*5 points*)

### Problem 2 - Decision trees [25 points]
***
The goal of this problem is to implement the core elements of the Decision Tree classifier.
We do not expect a highly efficient implementation of the functions since the ensemble methods will use the implementation
from scikit-learn.

We start by considering the variable *Colorado residency*.

The leaf nodes of a decision tree act in the same way as in question (1.1) where no feature is used.

2.1 Complete `get_error_in_leaf` to return the count of misclassified instances. (*3 points*)

In [89]:
def get_error_in_leaf(y, indices):
    """
    :param y: all labels
    :param indices: the subset of indexes in the leaf node
    :return: Returns the number of errors in a leaf node of a decision tree.
    """

    error_count = 0
    #BEGIN Workspace 2.1
    #TODO: Compute the number of errors in the leaf node (no feature is used)
    
    label_count = np.zeros(len(y))
    for index in indices:
        label_count[y[index]] += 1
        
    node_label = np.argmax(label_count)
    
    for index in indices:
        if y[index] != node_label:
            error_count += 1
    
    #END Workspace 2.1
    return error_count


def value_split_binary_feature(x, y, feature_index, root, criteria_func):
    """Will be used later to evaluate the criteria gain"""
    left_child = [i for i in root if x[i, feature_index] == 0]
    right_child = [i for i in root if x[i, feature_index] == 1]
    return criteria_func(y, root, left_child, right_child)


We will use information gain criteria to decide how to split the root node of our decision tree.

2.2 Complete the `entropy` function. (*5 points*)

2.3 Complete the `information_gain_criteria` to compute the information gained by splitting the root node.
 Print the gain value for splitting based on *Colorado residency* (*5 points*)


In [90]:
def entropy(y, indices):
    """
    :param y: all labels
    :param indices: the indices of data points
    :return: Returns the entropy in the labels for the data points in indices.
    """
    
    entropy_value = 0
    if len(indices) == 0: # deal with corner case when there is no data point.
        return entropy_value
    else:
        #BEGIN Workspace 2.2
        #TODO: Compute the entropy of the labels from indices
        
        n_classes = np.unique(y).shape[0]
        classes_counts = np.zeros(n_classes)
        for index in indices:
            classes_counts[labels[index]] += 1
        
        for num_of_class in classes_counts:
            if num_of_class > 0:
                prob = num_of_class/len(indices)
                entropy_value -= (prob)*np.log2(prob)
        
        #END Workspace 2.2
    return entropy_value

def information_gain_criteria(y, root, left_child, right_child):
    """
    :param y: all labels
    :param root: indices of all the data points in the root
    :param left_child: the subset of indices in the left child
    :param right_child: the subset of indices in the right child
    :return: information gain of the split
    """
    information_gain = 0
    #BEGIN Workspace 2.3.a
    #TODO: Compute the information gain of the split
    
    information_gain = entropy(y, root) - (len(left_child)/len(root))*entropy(y, left_child) - (len(right_child)/len(root))*entropy(y, right_child)
    
    #END Workspace 2.3.a
    return information_gain

In [91]:
feature_id = 2
info_gain = 0
#BEGIN Workspace 2.3.b
#TODO: report the information gain of the split based on Colorado Residency

print(f"information gain: {value_split_binary_feature(binary_features, labels, feature_id, np.arange(len(binary_features)), information_gain_criteria)}")

#END Workspace 2.3.b

information gain: 0.0912774462416801


Now we have to deal with continuous features for the decision tree.
One way to deal with continuous (or ordinal) data is to define binary features based on thresholding as we've done
for NaiveBayes. But we have to find the optimal threshold based on the criteria we're using.

2.4 Complete the `value_split_continuous_feature` by trying different possible threshold values of feature
of index `feature_index` and return the best criteria value and threshold. (*5 points*)

In [92]:
def value_split_continuous_feature(x, y, feature_index, root, criteria_func=information_gain_criteria):
    """
    :param x: all feature values
    :param y: all labels
    :param feature_index: feature id to split the tree based on
    :param root: indexes of all the data points in the root
    :param criteria_func: the splitting criteria function
    :return: Return the best value and its corresponding threshold by splitting based on a continuous feature.
    """

    best_value, best_thres = 0, 0

    #BEGIN Workspace 2.4
    #TODO: Complete the function as detailed in the question and function description
    
    for i in range(len(x)):
        curr_thres = x[i][feature_index]
        left_child = []
        right_child = []
        for j in range(len(x)):
            if x[j][feature_index] < curr_thres:
                left_child.append(j)
            else:
                right_child.append(j)
        curr_value = criteria_func(y, root, left_child, right_child)
        if curr_value > best_value:
            best_value = curr_value
            best_thres = curr_thres

    #END Workspace 2.4

    return best_value, best_thres

2.5 Find the best thresholds for age and salary. Print their corresponding information gains. (*5 points*)

In [93]:
root = list(range(len(labels))) # root includes all data points
#BEGIN Workspace 2.5
#TODO: Report the best thresholds for age and salary and their split information gains

best_age_value, best_age_threshold = value_split_continuous_feature(features, labels, 0, root)
best_salary_value, best_salary_threshold = value_split_continuous_feature(features, labels, 1, root)

print(f"Best age split threshold: {best_age_threshold}, corresponding information gain: {best_age_value}")
print(f"Best salary split threshold: {best_salary_threshold}, corresponding information gain: {best_salary_value}")

#END Workspace 2.5

Best age split threshold: 61, corresponding information gain: 0.1444843438056279
Best salary split threshold: 38000, corresponding information gain: 0.3219280948873623


2.6 Based on the obtained information gains, if we build a decision stump (decision tree with depth 1) greedily,
which feature should we choose? Why? What's the resulting accuracy? (*2 points*)

#BEGIN Workspace 2.6.a

We should choose the salary feature (specifically a 38,000 salary threshold) because it yields a higher information gain.

#END Workspace 2.6.a

In [97]:
#BEGIN Workspace 2.6.b
#TODO: Split based on the chosen feature and compute the accuracy (use get_error_in_leaf)
left_child, right_child = [], []
for i in range(len(features)):
    if features[i][1] < 38000:
        left_child.append(i)
    else:
        right_child.append(i)

left_error = get_error_in_leaf(labels, left_child) / len(left_child)
right_error = get_error_in_leaf(labels, right_child) / len(right_child)
total_error = (len(left_child)/len(features))*left_error + (len(right_child)/len(features))*right_error

print(f"Accuracy when splitting on a salary of 38,000: {100*(1 - total_error)}%")

#BEGIN Workspace 2.6.b

Accuracy when splitting on a salary of 38,000: 80.0%


**Bonus Question**

2.7 You now have all the ingredients to build a decision tree recursively.
You can build a decision tree of depth two and report its classification error on the training data and the tree.(*5 points*)

In [ ]:
#BEGIN Workspace 2.7
#TODO: Build a Decision Tree of Depth 2 using age, salary and the previously computed thresholds


#END Workspace 2.7

Problem 3  - Decision Tree Ensembles: Bagging and Boosting [50 points]
---

We are going to predict house price levels using decision tree ensembles.

In this classification problem, we compare Decision trees and it's ensembles - Bagging and Boosting on House Price prediction [dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

Our *weak learner* for this problem is the DecisionTreeClassifier from scikit-learn with `max_depth=10`.

We start first by loading preprocessed data that we'll use. Since the original data is for regression, we have first to transform
`y_train` and `y_test` to discrete values reflecting price level.

|Price range| Label|
|:----------:|--:|
| $ P < $125000|0|
|125000$\leq P < $ 160000| 1 |
|160000$ \leq P < $ 200000| 2 |
|200000$ \leq P $ | 3 |

3.1 Start by transforming`y_train` and `y_test` to discrete values using the provided ranges. (*3 points*)

In [99]:
X_train, X_test, y_train, y_test = pickle.load(open('./data/test_train.pkl','rb'))
#BEGIN Workspace 3.1
#TODO: Discretize y_train and y_test



#END Workspace 3.1
print(np.unique(y_train), X_train.shape)
print(np.unique(y_test), X_test.shape)

[ 34900  37900  52000  52500  55000  55993  58500  60000  61000  62383
  66500  72500  73000  75000  75500  76000  76500  78000  79000  79900
  80000  80500  81000  82000  82500  83000  84000  84500  85000  85400
  85500  86000  87000  87500  88000  89000  89500  90000  90350  91000
  91500  92900  93000  93500  94000  94500  94750  95000  96500  97000
  97500  98000  98300  98600  99500  99900 100000 101000 101800 102000
 103000 103600 104900 105000 105500 106000 106250 106500 107000 107400
 107500 107900 108000 108480 108500 108959 109000 109500 109900 110000
 110500 111000 111250 112000 112500 113000 114500 114504 115000 116000
 116050 116500 116900 117000 117500 118000 118400 118500 118858 118964
 119000 119500 119750 119900 120000 120500 121000 121500 121600 122000
 122500 122900 123000 123500 123600 124000 124500 124900 125000 125500
 126000 126175 126500 127000 127500 128000 128500 128900 129000 129500
 129900 130000 130500 131000 131400 131500 132000 132250 132500 133000
 13350

3.2 Complete the `ensemble_test` class to `fit` the model received as parameter and store the metrics and running time. (*5 points*)

3.3 Complete `plot_metric` to show and compare different statistics of each model in a bar chart. (*5 points*)

Later we will also use `ensemble_test` class to plot score, metric and time taken to fit the data.

In [ ]:
def get_weak_learner():
    """Return a new instance of our chosen weak learner"""
    return DecisionTreeClassifier(max_depth=10)

class EnsembleTest:
    """
        Test multiple model performance
    """

    def __init__(self, x_train, y_train, x_test, y_test):
        """
        initialize data partitions
        """
        self.scores = {}
        self.execution_time = {}
        self.metric = {}
        self.x_train = X_train
        self.y_train = y_train
        self.x_test = X_test
        self.y_test = y_test
        self.score_name ='Mean accuracy'
        self.metric_name = 'Precision(micro)'

    def fit_model(self, model, name):
        """
        Fit the model on train data.
        predict on test and store score and execution time for each fit.
        :param model: model
        :param name: name of model
        """
        start = time()
        #BEGIN Workspace 3.2
        #TODO: Fit the model and get the predictions
        #       train and test data are treated as global variables

        #Hint: self.scores[name] = precision_score(?, average="micro") # in multi-class, micro implies treating it as binary precision
        #Hint self.metric[name] = Accuracy
        #END Workspace 3.2
        self.execution_time[name] = time() - start

    def print_result(self):
        """
            print results for all models trained and tested.
        """
        models_cross = pd.DataFrame({
            'Model'         : list(self.metric.keys()),
             self.score_name     : list(self.scores.values()),
             self.metric_name    : list(self.metric.values()),
            'Execution time': list(self.execution_time.values())})
        print(models_cross.sort_values(by=self.score_name, ascending=False))

    def plot_metric(self):
        #BEGIN Workspace 3.3
        #TODO: plot bar chart for each metric : time, metric, score
        pass
        #END Workspace 3.3

3.4 Test `EnsembleTest` using our weak learner returned by `get_weak_learner` (*2 points*)

In [ ]:
# create a handler for ensemble_test, use the created handler for fitting different models.
ensemble_handler = EnsembleTest(X_train,y_train,X_test,y_test)
#BEGIN Workspace 3.4
#TODO: Initialize weak learner and fit ensemble_handler


#END Workspace 3.4
ensemble_handler.print_result()

**Bagging:**

Bagging consists of training a set of weak learners using random subsets of the train data.

3.5 First, complete `sample_data` to return a random sample of size `sample_ratio * len(X_train)` of features and labels (*2 points*)

3.6 Complete `fit` by building `n_estimators` of DecisionTreeClassifier, each trained on random sample of the data (*5 points*)

3.7 Complete `predict` method to return the most likely label by combining different estimators predictions. 
Use a simple majority / plurality vote system similar to the one used in your KNNClassifier in Problem Set 1. However, in this case, to break a tie you should use `predict_log_proba` or `predict_proba` method of DecisionTreeClassifier:
[Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.predict_proba) (*2 points*)

In [ ]:
class BaggingEnsemble(object):

    def __init__(self, n_estimators, sample_ratio):
        self.n_estimators = n_estimators
        self.sample_ratio = sample_ratio
        self.estimators = []

    def sample_data(self, x_train, y_train):
        x_sample, y_sample = None, None
        #BEGIN Workspace 3.5
        #TODO: sample random subset of size sample_ratio * len(X_train)
        
        
        #END Workspace 3.5
        return x_sample, y_sample

    def fit(self, x_train, y_train):
        for _ in range(self.n_estimators):
            #BEGIN Workspace 3.6
            #TODO: sample data and create new weak learned trained on the sample
            pass
            #END Workspace 3.6

    def predict(self, X_test):
        predicted_proba = 0
        answer = 0
        #BEGIN Workspace 3.7
        #TODO: go through the trained estimators and accumulate their predicted_proba to get the mostly likely label
        pass
        #END Workspace 3.7
        return answer


In [ ]:
# This cell should run without errors
ensemble_handler.fit_model(BaggingEnsemble(10, 0.9),'Bagging')
ensemble_handler.print_result()


**Random Forest**
Random Forest has an additional layer of randomness compared to Bagging: we also sample a random subset of the features.

3.8 First, complete `sample_data` to return a random sample of size `sample_ratio * len(X_train)` of labels and `feature_ratio * num_features` of features (*2 points*)

3.9 Complete `fit` by building `n_estimators` of DecisionTreeClassifier, each trained on random sample of the data.
Make sure to keep track of the sampled features for each estimator to use them in the prediction step. (*5 points*)

3.10 Complete `predict` method to return the most likely label by combining different estimators predictions.
Use a simple majority / plurality vote system similar to the one used in your KNNClassifier in Problem Set 1. However, in this case, to break a tie you should use `predict_log_proba` or `predict_proba`  method of DecisionTreeClassifier:
[Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.predict_proba) (*3 points*)


In [ ]:
class RandomForest(object):

    def __init__(self, n_estimators, sample_ratio, features_ratio):
        self.n_estimators = n_estimators
        self.sample_ratio = sample_ratio
        self.features_ratio = features_ratio
        self.estimators = []
        self.features_indices = []

    def sample_data(self, x_train, y_train):
        X_sample, y_sample, features_indices = None, None, None
        #BEGIN Workspace 3.8
        #TODO: sample random subset of size sample_ratio * len(X_train) and subset of features of size
        #         features_ratio * num_features


        #END Workspace 3.8
        return x_sample, y_sample, features_indices

    def fit(self, x_train, y_train):
        for _ in range(self.n_estimators):
            #BEGIN Workspace 3.9
            #TODO: sample data with random subset of rows and features using sample_data
            #Hint: keep track of the features indices in features_indices to use in predict
            pass
            #END Workspace 3.9

    def predict(self, X_test):
        predicted_proba = 0
        answer = 0
        #BEGIN Workspace 3.10
        #TODO: compute cumulative sum of predict proba from estimators and return the labels with highest likelihood

        #END Workspace 3.10
        return answer

In [ ]:
# This cell should run without errors
ensemble_handler.fit_model(RandomForest(20, sample_ratio=0.9, features_ratio=0.8), 'RandomForest')
ensemble_handler.print_result()

**Boosting**

There are different methods of boosting, but we'll focus in this problem on Adaptive Boosting (AdaBoost).
The logic of AdaBoost is to "push" the new learner to give more importance to previously misclassified data. We present
below the multiclass variant of AdaBoost [SAMME](https://web.stanford.edu/~hastie/Papers/samme.pdf). We denote $K$ the number of classes.

AdaBosst is performed by increasing the weights of misclassified simple after each iteration:
- Start with equal weights $W_1 = (w_i), $ where   $w_i = \frac{1}{\texttt{n_samples}}$
- at step j:
    - Train estimator $h_j$ using weights $W_j$
    - Find the weighted error rate $\epsilon_j$ using $W_j$: $\epsilon_j=\frac{\sum_i w_i \delta(\hat{y}_i, y_i)}{\sum_i w_i}$
    - Choose $\alpha_j = \log \frac{1-\epsilon_j}{\epsilon_j} + \log(K-1)$
    - Update $W_j$ using: $w_i \leftarrow w_i \exp(\alpha_j \delta(\hat{y_i}, y_i)) $
    - Normalize $W_j$ to have sum 1
- Global estimator is $H = \sum_j \alpha_j h_j$

$\hat{y}$ are the predicted labels, and $\delta$ the Kronecker function, equal to 1 when the two argument are equal, 0 otherwise.


3.11 Complete `fit` by building `n_estimators` of DecisionTreeClassifier, each trained on the same data but with different
samples weights as detailed in the algorithm. Keep track of $(\alpha_i)$ (*7 points*)

3.12 Complete `predict` method to return the predicted label using the global estimator $H$ (*5 points*). Hint:
use one hot encoding of the predicted labels from the weak learners and cumulate the prediction with weights $\alpha_j$ 

Notice that if the weak learner is consistent (0 error rate on the training set), AdaBoost $\alpha_j$ are no longer defined.

In [ ]:
class AdaBoost(object):

    def __init__(self, n_estimators, num_classes=4):
        self.n_estimators = n_estimators
        self.num_classes = num_classes
        self.estimators = []
        self.alphas = []
        self.weights = None


    def fit(self, X_train, y_train):
        #BEGIN Workspace 3.11
        #TODO: Implement Multiclass Adaboost and keep track of the alpha_j
        pass
        #END Workspace 3.11

    def predict(self, X_test):
        answer = 0
        #BEGIN Workspace 3.12
        #TODO: get the labels returned by the global estimator defined as H
        #Hint: Use one-hot format to get the labels using the global estimator
        #Hint: We don't need predict_proba for this one
        
        #END Workspace 3.12
        return answer

In [ ]:
# This cell should run without errors
ensemble_handler.fit_model(AdaBoost(10), 'AdaBoost')
ensemble_handler.print_result()

**Comparison**

3.13 Add different ensemble methods to the handler (try different parameters), plot, show, and compare them (*5 points*).

In [ ]:
# create a handler for ensemble_test, use the created handler for fitting different models.
ensemble_handler = EnsembleTest(X_train, y_train, X_test, y_test)
decision = get_weak_learner()
ensemble_handler.fit_model(decision,'decision_tree')
#BEGIN Workspace 3.13.a
#TODO Add multiple instances of the ensemble methods. Plot and compare their performance

#END Workspace 3.13.a

#BEGIN Workspace 3.13.b
#TODO Comparison write-up
#END Workspace 3.13.b